In [3]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install TA-lib
!pip install numpy pandas matplotlib yfinance mplfinance

import yfinance as yf
import talib as ta
import pandas as pd
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output

--2024-11-29 15:42:02--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:c95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.13.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-11-29 15:42:02--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:d95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://gigenet.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-11-29 15:42:02--  http://gigenet.dl.source

In [9]:
pattern_functions = {
    'Crows': ta.CDL2CROWS,
    'Inside': ta.CDL3INSIDE,
    'Abandoned Baby': ta.CDLABANDONEDBABY,
    'Advance Block': ta.CDLADVANCEBLOCK,
    'Break Away': ta.CDLBREAKAWAY,
    'Counter Attack': ta.CDLCOUNTERATTACK,
    'Dark Cloud Cover': ta.CDLDARKCLOUDCOVER,
    'Doji': ta.CDLDOJI,
    'Doji Star': ta.CDLDOJISTAR,
    'Dragon Fly Doji': ta.CDLDRAGONFLYDOJI,
    'Engulfing': ta.CDLENGULFING,
    'Evening Doji Star': ta.CDLEVENINGDOJISTAR,
    'Evening Star': ta.CDLEVENINGSTAR,
    'Grave Stone Doji': ta.CDLGRAVESTONEDOJI,
    'Hammer': ta.CDLHAMMER,
    'Hanging Man': ta.CDLHANGINGMAN,
    'Harami': ta.CDLHARAMI,
    'High Wave': ta.CDLHIGHWAVE,
    'Hikkake': ta.CDLHIKKAKE,
    'Inverted Hammer': ta.CDLINVERTEDHAMMER,
    'Kicking': ta.CDLKICKING,
    'Ladder Bottom': ta.CDLLADDERBOTTOM,
    'Matching Glow': ta.CDLMATCHINGLOW,
    'Morning Doji Star': ta.CDLMORNINGDOJISTAR,
    'Morning Star': ta.CDLMORNINGSTAR,
    'Shooting Star': ta.CDLSHOOTINGSTAR,
    'Spinning Top': ta.CDLSPINNINGTOP,
    'Stalled Pattern': ta.CDLSTALLEDPATTERN,
    'Stick Sandwich': ta.CDLSTICKSANDWICH,
    'Tristar': ta.CDLTRISTAR
}

ticker_label = widgets.Label(value="Ticker Symbol")
ticker_input = widgets.Text(placeholder="Enter ticker symbol", style={'description_width': 'initial'})

start_date_label = widgets.Label(value="Start Date")
start_date_input = widgets.DatePicker(description="Start Date", style={'description_width': 'initial'})

end_date_label = widgets.Label(Value="End Date")
end_date_input = widgets.DatePicker(description="End Date", style={'description_width': 'initial'})

pattern_label = widgets.Label(value="Pattern")
pattern_input = widgets.Dropdown(description="Pattern", options=list(pattern_functions.keys()), style={'description_width': 'initial'})

plot_button = widgets.Button(description="Plot the Candlestick PatternChart", button_style='success')

output_area = widgets.Output()

ui = VBox([
    HBox([ticker_label, ticker_input], layout=widgets.Layout(margin="15px 0")),
    HBox([start_date_label, start_date_input], layout=widgets.Layout(margin="15px 0")),
    HBox([end_date_label, end_date_input], layout=widgets.Layout(margin="15px 0")),
    HBox([pattern_label, pattern_input], layout=widgets.Layout(margin="15px 0")),
    plot_button,
    output_area
])

def plot_candlestick_for_pattern(data, pattern, ticker):
    if pattern not in data.columns:
        with output_area:
            print(f"Pattern {pattern} not found in data.")
        return

    filtered_data = data[data[pattern] != 0]

    if filtered_data.empty:
        with output_area:
            print(f"No data found for the selected pattern: {pattern}.")
        return

    filtered_data = filtered_data.reset_index()
    filtered_data['Date'] = mdates.date2num(filtered_data['Date'])
    ohlc = filtered_data[['Date', 'Open', 'High', 'Low', 'Close']].values

    fig, ax = plt.subplots(figsize=(16, 20))
    candlestick_ohlc(ax, ohlc, width=20, colorup='green', colordown='red', alpha=0.8)

    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.set_title(f"Candlestick Chart for {pattern} in {ticker} stock", fontsize=18)
    ax.set_xlabel("Date", fontsize=14)
    ax.set_ylabel("Price", fontsize=14)
    ax.grid(color='gray', linestyle='--', linewidth=0.5)

    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    pdf_filename = f"{ticker}_{pattern}.pdf"
    plt.savefig(pdf_filename, format='pdf', bbox_inches='tight')
    print(f"Chart saved")
    plt.show()

def on_plot_button_clicked(b):
    clear_output(wait=True)
    with output_area:
        output_area.clear_output()

        # Get user inputs
        ticker = ticker_input.value
        start_date = start_date_input.value
        end_date = end_date_input.value
        pattern = pattern_input.value

        if not ticker or not start_date or not end_date:
            print("Please fill in all fields.")
            return

        start_date = start_date.strftime("%Y-%m-%d")
        end_date = end_date.strftime("%Y-%m-%d")

        # Fetch data
        print(f"Fetching data for {ticker} from {start_date} to {end_date}...")
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            print("No data available for the given date range.")
            return

        data = data.drop(columns=['Adj Close', 'Volume'], errors='ignore')
        data[['Open', 'High', 'Low', 'Close']] = data[['Open', 'High', 'Low', 'Close']].apply(pd.to_numeric, errors='coerce')

        data['Open'] = pd.to_numeric(data['Open'].values.reshape(-1), errors='coerce')
        data['High'] = pd.to_numeric(data['High'].values.reshape(-1), errors='coerce')
        data['Low'] = pd.to_numeric(data['Low'].values.reshape(-1), errors='coerce')
        data['Close'] = pd.to_numeric(data['Close'].values.reshape(-1), errors='coerce')

        open_prices = data['Open'].values.reshape(-1)
        high_prices = data['High'].values.reshape(-1)
        low_prices = data['Low'].values.reshape(-1)
        close_prices = data['Close'].values.reshape(-1)

        pattern_function = pattern_functions[pattern_input.value]
        data[pattern] = pattern_function(open_prices, high_prices, low_prices, close_prices)

        plot_candlestick_for_pattern(data, pattern, ticker)

plot_button.on_click(on_plot_button_clicked)

display(ui)